# IMDB(TensorFlow版本)

In [25]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, SimpleRNN
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist
import tensorflow as tf

total_words = 10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=total_words)

In [26]:
max_review_len = 80
# x_train:[b, 80]
# x_test: [b, 80]
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

In [27]:
x_train[:5]

array([[  15,  256,    4,    2,    7, 3766,    5,  723,   36,   71,   43,
         530,  476,   26,  400,  317,   46,    7,    4,    2, 1029,   13,
         104,   88,    4,  381,   15,  297,   98,   32, 2071,   56,   26,
         141,    6,  194, 7486,   18,    4,  226,   22,   21,  134,  476,
          26,  480,    5,  144,   30, 5535,   18,   51,   36,   28,  224,
          92,   25,  104,    4,  226,   65,   16,   38, 1334,   88,   12,
          16,  283,    5,   16, 4472,  113,  103,   32,   15,   16, 5345,
          19,  178,   32],
       [ 125,   68,    2, 6853,   15,  349,  165, 4362,   98,    5,    4,
         228,    9,   43,    2, 1157,   15,  299,  120,    5,  120,  174,
          11,  220,  175,  136,   50,    9, 4373,  228, 8255,    5,    2,
         656,  245, 2350,    5,    4, 9837,  131,  152,  491,   18,    2,
          32, 7464, 1212,   14,    9,    6,  371,   78,   22,  625,   64,
        1382,    9,    8,  168,  145,   23,    4, 1690,   15,   16,    4,
        135

In [28]:
y_train[:5]

array([1, 0, 0, 1, 0], dtype=int64)

In [29]:
batch_size = 32
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.shuffle(1000).batch(batch_size, drop_remainder=True)
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.batch(batch_size, drop_remainder=True)

In [30]:
embedding_len =32

class MyRNN(tf.keras.Model):

    def __init__(self, units):
        super(MyRNN, self).__init__()

        # [b, 64]
        self.state = [tf.zeros([batch_size, units])]
        # self.state1 = [tf.zeros([batchsz, units])]
        # transform text to embedding representation
        # [b, 80] => [b, 80, 32]
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len)
        # [b, 80, 32] , h_dim: 64
        # RNN: cell1 ,cell2, cell3
        # SimpleRNN
        self.rnn_cell = tf.keras.layers.SimpleRNNCell(units, dropout=0.2)
        # self.rnn_cell1 = layers.SimpleRNNCell(units, dropout=0.5)
        # fc, [b, 80, 32] => [b, 64] => [b, 1]
        self.fc= tf.keras.layers.Dense(1) # 二元分類也可以輸出層只給1個神經元，但後面的loss要用binary_cross_entropy

    def call(self, inputs, training=None):
        # [b, 80]
        x = inputs
        # embedding: [b, 80] => [b, 80, 32]
        x = self.embedding(x)
        # rnn cell compute
        # [b, 80, 32] => [b, 64]
        state = self.state
        # state1 = self.state1
        for word in tf.unstack(x, axis=1): # word: [b, 32]
            # h1 = x*wxh+h0*whh
            # out: [b, 64]
            out, state = self.rnn_cell(word, state, training)
        # out: [b, 64] => [b, 1]
        x = self.fc(out)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob

model = MyRNN(64) 
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
history = model.fit(db_train,
                    epochs=5,
                    batch_size=512,
                    validation_data=db_test,
                    verbose=1)

Epoch 1/5
781/781 [==============================] - 4s 5ms/step - loss: 0.5939 - accuracy: 0.6612 - val_loss: 0.4388 - val_accuracy: 0.8050
Epoch 2/5
781/781 [==============================] - 3s 4ms/step - loss: 0.3700 - accuracy: 0.8389 - val_loss: 0.4240 - val_accuracy: 0.8220
Epoch 3/5
781/781 [==============================] - 3s 4ms/step - loss: 0.2639 - accuracy: 0.8932 - val_loss: 0.4769 - val_accuracy: 0.8154
Epoch 4/5
781/781 [==============================] - 4s 5ms/step - loss: 0.1814 - accuracy: 0.9309 - val_loss: 0.5956 - val_accuracy: 0.7567
Epoch 5/5
781/781 [==============================] - 3s 4ms/step - loss: 0.1196 - accuracy: 0.9554 - val_loss: 0.6436 - val_accuracy: 0.7958
